# Exploratory Notebook

This notebook is a starter exploratory environment for the Hybrid Language Music Clustering project.
Populate data/audio and data/lyrics, then run preprocessing and model training cells.

In [ ]:
# Basic imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
# Add project src to path if needed
import sys
sys.path.append(os.path.abspath('../src'))